## Library

In [1]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
from math import sqrt
import pickle 

from collections import Counter
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression



## Loading Data 

In [2]:
D = np.load('tf_idf_vector_matrix.npy') # load

In [3]:
D.shape

(9025, 22336)

In [4]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [5]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 26 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

In [6]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",1995
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,When siblings Judy and Peter discover an encha...,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,A family wedding reignites the ancient feud be...,1995
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Just when George Banks has recovered from his ...,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9020,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"['Comedy', 'Horror', 'Science Fiction']",http://www.syfy.com/sharknado4,390989,tt4831420,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,...,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What happens in Vegas, stays in Vegas. Unless ...",Sharknado 4: The 4th Awakens,False,4.3,88.0,The new installment of the Sharknado franchise...,2016
9021,False,NaN,8000000,['Drama'],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,A man must cope with the loss of his wife and ...,2001
9022,False,NaN,1000000,"['Thriller', 'Romance']",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"Rustom Pavri, an honourable officer of the Ind...",2016
9023,False,NaN,15050000,"['Adventure', 'Drama', 'History', 'Romance']",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,155.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"Village lad Sarman is drawn to big, bad Mohenj...",2016


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99810 entries, 0 to 99809
Data columns (total 6 columns):
userId        99810 non-null int64
id            99810 non-null float64
rating        99810 non-null float64
movieId       99810 non-null int64
timestamp     99810 non-null int64
movieIndex    99810 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB


In [8]:
ratings

,userId,id,rating,movieId,timestamp,movieIndex
0,0,152.0,2.5,1371,1260759135,1107
1,0,9909.0,2.5,31,1260759144,30
2,0,847.0,2.0,2193,1260759198,1739
3,0,9426.0,2.5,2455,1260759113,1958
4,0,6114.0,3.5,1339,1260759125,1079
...,...,...,...,...,...,...
99805,670,786.0,2.0,3897,1063503718,3108
99806,670,7443.0,4.0,3751,1065111939,2995
99807,670,1891.0,5.0,1196,1064890635,949
99808,670,279.0,4.0,1225,1065149143,977


In [9]:
train.shape

(79848, 6)

In [10]:
test.shape

(19962, 6)

In [11]:
n_users = len(ratings['userId'].unique().astype('int'))
n_items = len(ratings['id'].unique().astype('int'))
n_users

671

In [12]:
len(train[train['userId'] == 1])

56

In [13]:
len(test[test['userId'] == 1])

19

In [14]:
ratings_train = train[['userId','id','rating','movieIndex']].values
ratings_test = test[['userId','id','rating','movieIndex']].values

In [15]:
ratings_train.shape

(79848, 4)

In [16]:
ratings_test.shape

(19962, 4)

In [17]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [18]:
#a, b = get_items_rated_by_user(ratings_train, 670)
#for i in a:
#    print(i)

In [19]:
D.shape

(9025, 22336)

In [20]:
d = D.shape[1] # movie vector dimention
W = np.zeros((d, n_users)) 
b = np.zeros((1,n_users))

In [21]:
for i in range(n_users):
        idx, scores = get_items_rated_by_user(ratings_train, i)
        clf = Ridge(alpha=1.0, random_state=241)
        #idx = np.asarray(idx)
        Xhat = D[idx,:]
        
        clf.fit(Xhat, scores)
        W[:, i] = clf.coef_
        b[0, i] = clf.intercept_

In [22]:
W

array([[ 0.        ,  0.03023679,  0.01808287, ..., -0.00677314,
         0.03899125,  0.02888918],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.02827725,  0.20555655],
       [ 0.        ,  0.        ,  0.05487934, ..., -0.06578338,
         0.07093938,  0.19235286],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [23]:
Yhat = D.dot(W) + b
Yhat.shape

(9025, 671)

In [24]:
type(Yhat)

numpy.ndarray

In [25]:
n = 0
ids, scores = get_items_rated_by_user(ratings_test, n)
result = Yhat[ids, n]

In [26]:
print("ratings prediction of user ",n," :")
for i in range(len(result)):
    print("movie index : ",ids[i]," - ","predict ratings : ",result[i]," - ","true ratings : ",scores[i])

ratings prediction of user  0  :
movie index :  30  -  predict ratings :  2.426854460256589  -  true ratings :  2.5
movie index :  1739  -  predict ratings :  2.423113389081579  -  true ratings :  2.0
movie index :  1661  -  predict ratings :  2.4192961861067674  -  true ratings :  4.0
movie index :  927  -  predict ratings :  2.4315037074263337  -  true ratings :  4.0
movie index :  903  -  predict ratings :  2.421217296468647  -  true ratings :  2.0


In [27]:
#train[train['userId'] == 0]

In [28]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

In [29]:
print('RMSE for train case:', evaluate(Yhat, ratings_train, W, b))
print('RMSE for test case:', evaluate(Yhat, ratings_test, W, b))

RMSE for train case: 0.7666285240992233
RMSE for test case: 0.953144781499417


In [30]:
test

,userId,id,rating,movieId,timestamp,movieIndex
0,0,9909.0,2.5,31,1260759144,30
1,0,847.0,2.0,2193,1260759198,1739
2,0,97.0,4.0,2105,1260759139,1661
3,0,11216.0,4.0,1172,1260759205,927
4,0,1103.0,2.0,1129,1260759185,903
...,...,...,...,...,...,...
19957,670,11310.0,2.5,432,1063503739,381
19958,670,712.0,3.0,357,1063503998,322
19959,670,630.0,4.0,919,1065149458,737
19960,670,687.0,4.0,36,1065149314,34


In [31]:
np.save('tf_idf_pred_rating.npy', Yhat)

In [32]:
predict_rating = []
for n in range(n_users):
    ids, scores_truth = get_items_rated_by_user(ratings_test, n)
    for i in ids:
        scores_pred = Yhat[i, n]
        predict_rating.append(scores_pred)           
        


In [36]:
f = open('tf_idf_pred_rating.pckl', 'wb')
pickle.dump(predict_rating, f)
f.close()

In [33]:
test['TF-IDF pred'] = predict_rating

In [34]:
test

,userId,id,rating,movieId,timestamp,movieIndex,TF-IDF pred
0,0,9909.0,2.5,31,1260759144,30,2.426854
1,0,847.0,2.0,2193,1260759198,1739,2.423113
2,0,97.0,4.0,2105,1260759139,1661,2.419296
3,0,11216.0,4.0,1172,1260759205,927,2.431504
4,0,1103.0,2.0,1129,1260759185,903,2.421217
...,...,...,...,...,...,...,...
19957,670,11310.0,2.5,432,1063503739,381,3.872884
19958,670,712.0,3.0,357,1063503998,322,3.845205
19959,670,630.0,4.0,919,1065149458,737,3.941357
19960,670,687.0,4.0,36,1065149314,34,3.878385


In [35]:
Yhat[381,670]

3.8728841163963232